### cabeçalho

#### importações

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

#### lendo as bases

In [ ]:
URL_BASES = "./../Databases/test/"
patient_csv = "patient.csv"
infusiondrug = "infusiondrug.csv"
physicalExam = "physicalExam.csv"
lab = "lab.csv"

url_patient = URL_BASES + patient_csv
url_infusiondrug = URL_BASES + infusiondrug
url_physicalExam = URL_BASES + physicalExam
url_lab = URL_BASES + lab

comeco_intervalo = 60 * 0
final_intervalo = 60 * 24

#### pacientes septicos

In [ ]:
df_patient = pd.read_csv(url_patient)
sepsis = df_patient[df_patient['apacheadmissiondx'].str.contains('sepsis', case=False, na=False)].reset_index()
sepsis = sepsis[['patientunitstayid','gender','age']]
del df_patient
sepsis.head()

### tudo sobre physical exam

#### 24 horas physical exam

In [ ]:
df_physicalExam = pd.read_csv(url_physicalExam)
df_physicalExam_24 = df_physicalExam[(df_physicalExam["physicalexamoffset"]>=comeco_intervalo) &  (df_physicalExam["physicalexamoffset"]<=final_intervalo)]
df_physicalExam_24 = pd.merge(sepsis['patientunitstayid'],df_physicalExam_24,on='patientunitstayid')
del df_physicalExam

#### cada feature individual

In [ ]:
def filter_exam(df, exam_name):
    return df[df['physicalexampath'].str.contains(exam_name, case=False, na=False)]

motor = filter_exam(df_physicalExam_24,'Motor Score')
eyes = filter_exam(df_physicalExam_24,'Eyes Score')
verbal = filter_exam(df_physicalExam_24,'Verbal Score')
systolic = filter_exam(df_physicalExam_24,'(systolic)')
systolic = filter_exam(systolic,'Current')
diastolic = filter_exam(df_physicalExam_24,'(diastolic)')
diastolic = filter_exam(diastolic,'Current')
ox_saturation = filter_exam(df_physicalExam_24,'O2 Sat%')
ox_saturation = filter_exam(ox_saturation,'Current')
heartRate = filter_exam(df_physicalExam_24,'HR Current')

del df_physicalExam_24

#### editando as features

In [ ]:
def edit_feature_exam(df,exam_name):
    df.drop(['physicalexamid', 'physicalexampath', 'physicalexamvalue'], axis=1, inplace=True)
    df.rename(columns={'physicalexamtext': exam_name}, inplace = True)

edit_feature_exam(motor,'motor')
edit_feature_exam(eyes,'eyes')
edit_feature_exam(verbal,'verbal')
edit_feature_exam(systolic,'systolic')
edit_feature_exam(diastolic,'diastolic')
edit_feature_exam(ox_saturation,'ox_saturation')
edit_feature_exam(heartRate,'heartRate')

#### merge

In [ ]:

df_physicalExam_24 = pd.merge(motor, eyes, on=['patientunitstayid','physicalexamoffset'],how='outer')
df_physicalExam_24 = pd.merge(df_physicalExam_24, verbal, on=['patientunitstayid','physicalexamoffset'],how='outer')
df_physicalExam_24 = pd.merge(df_physicalExam_24, systolic, on=['patientunitstayid','physicalexamoffset'],how='outer')
df_physicalExam_24 = pd.merge(df_physicalExam_24, diastolic, on=['patientunitstayid','physicalexamoffset'],how='outer')
df_physicalExam_24 = pd.merge(df_physicalExam_24, ox_saturation, on=['patientunitstayid','physicalexamoffset'],how='outer')
df_physicalExam_24 = pd.merge(df_physicalExam_24, heartRate, on=['patientunitstayid','physicalexamoffset'],how='outer')

del motor
del verbal
del eyes
del systolic
del diastolic
del ox_saturation
del heartRate

In [ ]:
df_physicalExam_24.head(50)

#### Mean arterial presure

In [ ]:
# Tenta converter colunas para inteiro e usa pd.to_numeric com errors='coerce' em caso de falha
df_physicalExam_24["diastolic"] = pd.to_numeric(df_physicalExam_24["diastolic"], errors='coerce')
df_physicalExam_24["systolic"] = pd.to_numeric(df_physicalExam_24["systolic"], errors='coerce')

# Calcula o MAP (Mean Arterial Pressure) ignorando automaticamente valores nulos
df_physicalExam_24['MAP'] = (df_physicalExam_24['systolic'] + 2 * df_physicalExam_24['diastolic']) / 3


#### Glasgow Coma Scale

In [ ]:
df_physicalExam_24['GCS'] = df_physicalExam_24['motor'] + df_physicalExam_24['verbal'] + df_physicalExam_24['eyes']

In [ ]:
df_physicalExam_24.to_csv((URL_BASES+'physicalexam24.csv'),index=False)

### tudo sobre lab

#### 24 horas lab

In [ ]:
df_lab = pd.read_csv(url_lab)
df_lab_24 = df_lab.loc[(df_lab["labresultoffset"] >= comeco_intervalo) & (df_lab["labresultoffset"] <= final_intervalo)]
df_lab_24 = pd.merge(sepsis['patientunitstayid'],df_lab_24,on='patientunitstayid')
df_lab_24.sort_values(by=['patientunitstayid','labresultoffset'],inplace=True)
del df_lab

def filter_lab(df, lab_name):
    return df[df['labname'].str.contains(lab_name, case=False, na=False)]

def edit_feature_lab(df,lab_name):
    df.drop(['labid','labtypeid','labname','labresultrevisedoffset'],axis=1,inplace=True)
    df.rename(columns={'labresult': lab_name, 'labresulttext' : (lab_name+'resulttext'), 'labmeasurenamesystem' : (lab_name+'.unidMedida'), 'labmeasurenameinterface': (lab_name+'.measureInterface')},inplace=True)


#### inflamatory

In [ ]:
bands = filter_lab(df_lab_24, 'bands')
CRP = filter_lab(df_lab_24, 'CRP')
ESR = filter_lab(df_lab_24, 'ESR')
Temperature = filter_lab(df_lab_24, 'Temp')
WBC = filter_lab(df_lab_24, 'WBC x 1000')

edit_feature_lab(bands,'bands')
edit_feature_lab(CRP,'CRP')
edit_feature_lab(ESR,'ESR')
edit_feature_lab(Temperature,'Temperature')
Temperature['celsius'] = 0
Temperature['Temperatureresulttext'] = Temperature['Temperatureresulttext'].astype(float)
Temperature['celsius'] = Temperature['Temperatureresulttext'].map(lambda x: x if x < 60 else (x - 32) * 5/9)
edit_feature_lab(WBC,'WBC')

In [ ]:
inflamatory = pd.merge(bands,CRP, on=['patientunitstayid','labresultoffset'], how='outer')
inflamatory = pd.merge(inflamatory,ESR, on=['patientunitstayid','labresultoffset'], how='outer')
inflamatory = pd.merge(inflamatory,Temperature, on=['patientunitstayid','labresultoffset'], how='outer')
inflamatory = pd.merge(inflamatory,WBC, on=['patientunitstayid','labresultoffset'], how='outer')
inflamatory.head()

#### Renal

In [ ]:
BUN = filter_lab(df_lab_24, 'BUN')
creatinine = filter_lab(df_lab_24, 'creatinine')

edit_feature_lab(BUN,'BUN')
edit_feature_lab(creatinine,'creatinine')

In [ ]:
Renal = pd.merge(BUN,creatinine,on=['patientunitstayid','labresultoffset'],how='outer')
Renal.head()

#### cardiac

In [ ]:
bicarbonate = filter_lab(df_lab_24, 'bicarbonate')
lactate = filter_lab(df_lab_24, 'lactate')
troponin = filter_lab(df_lab_24, 'troponin')

edit_feature_lab(bicarbonate,'bicarbonate')
edit_feature_lab(lactate,'lactate')
edit_feature_lab(troponin,'troponin')

In [ ]:
Cardiac = pd.merge(bicarbonate,lactate,on=['patientunitstayid','labresultoffset'],how='outer')
Cardiac = pd.merge(Cardiac,troponin,on=['patientunitstayid','labresultoffset'],how='outer')
Cardiac.head()

#### Pulmonary

In [ ]:
pao2 = filter_lab(df_lab_24, 'paO2')
respiratory_rate = filter_lab(df_lab_24, 'Respiratory Rate')

edit_feature_lab(pao2,'pao2')
edit_feature_lab(respiratory_rate, 'RR')

In [ ]:
Pulmonary = pd.merge(pao2,respiratory_rate,on=['patientunitstayid','labresultoffset'],how='outer')
Pulmonary.head()

#### Hepatic

In [ ]:
alanine = filter_lab(df_lab_24, 'ALT')
aspartate = filter_lab(df_lab_24, 'AST')
bilirubin = filter_lab(df_lab_24, 'bilirubin')

edit_feature_lab(bilirubin,'bilirubin')
edit_feature_lab(alanine,'alanine')
edit_feature_lab(aspartate,'aspartate')

In [ ]:
Hepatic = pd.merge(alanine,aspartate,on=['patientunitstayid','labresultoffset'],how='outer')
Hepatic = pd.merge(Hepatic, bilirubin,on=['patientunitstayid','labresultoffset'],how='outer')

Hepatic.head()

#### Hematologic

In [ ]:
hemoglobin = filter_lab(df_lab_24, 'Hgb')
inr = filter_lab(df_lab_24, 'INR')
platelets = filter_lab(df_lab_24, 'platelets')

edit_feature_lab(hemoglobin,'hemoglobin')
edit_feature_lab(inr,'inr')
edit_feature_lab(platelets,'platelets')

In [ ]:
Hematologic = pd.merge(hemoglobin,inr,on=['patientunitstayid','labresultoffset'],how='outer')
Hematologic = pd.merge(Hematologic,platelets,on=['patientunitstayid','labresultoffset'],how='outer')

Hematologic.head()

#### Serum levels

In [ ]:
albumin = filter_lab(df_lab_24, 'albumin')
chloride = filter_lab(df_lab_24, 'chloride')
glucose = filter_lab(df_lab_24, 'glucose')
sodium = filter_lab(df_lab_24, 'sodium')

edit_feature_lab(albumin,'albumin')
edit_feature_lab(chloride,'chloride')
edit_feature_lab(glucose,'glucose')
edit_feature_lab(sodium,'sodium')

In [ ]:
SerumLevel = pd.merge(albumin,chloride,on=['patientunitstayid','labresultoffset'],how='outer')
SerumLevel = pd.merge(SerumLevel,glucose,on=['patientunitstayid','labresultoffset'],how='outer')
SerumLevel = pd.merge(SerumLevel,sodium,on=['patientunitstayid','labresultoffset'],how='outer')

SerumLevel.head()

#### lab final

In [ ]:
df_lab_24.head(10000)

### tudo sobre infusion drug

In [ ]:
df_infusiondrug = pd.read_csv(url_infusiondrug)
df_infusiondrug_24 = df_infusiondrug[(df_infusiondrug["infusionoffset"]>=comeco_intervalo) &  (df_infusiondrug["infusionoffset"]<=final_intervalo)]
df_infusiondrug_24 = pd.merge(sepsis['patientunitstayid'],df_infusiondrug_24,on='patientunitstayid')
del df_infusiondrug

In [ ]:
dopamine = df_infusiondrug_24[df_infusiondrug_24['drugname'].str.contains('Dopamine', case=False, na=False)]
dobutamine = df_infusiondrug_24[df_infusiondrug_24['drugname'].str.contains('Dobutamine', case=False, na=False)]
epinephrine = df_infusiondrug_24[df_infusiondrug_24['drugname'].str.contains('Epinephrine', case=False, na=False)]
norepinephrine = df_infusiondrug_24[df_infusiondrug_24['drugname'].str.contains('Norepinephrine', case=False, na=False)]

del df_infusiondrug_24

#### editando as features